In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('./utils')
sys.path.append('./templates')

from common_lib import check_packages
check_packages()


Proceed.


In [2]:
import os
import json
import boto3
import torch
import sagemaker, boto3, jinja2
from sagemaker import get_execution_role
from sagemaker.model import Model
from sagemaker.estimator import Estimator
from sagemaker import serializers, deserializers
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


<br>

## 1. Local에서 학습한 PEFT 모델을 로드
---

In [11]:
%store -r output_dir

In [12]:
local_peft_model_dir = output_dir

In [13]:
local_peft_model_dir

'output'

In [14]:
torch.cuda.empty_cache()

### Model 환경설정

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

### 만약 아래 로컬 학습 진행한 output 폴더에 adapter_config.json 파일의 경로가 없을 경우 output 폴더를 삭제하고 로컬 학습 과정을 순서대로 진행해주세요. 

In [22]:
peft_model_id = local_peft_model_dir
config = PeftConfig.from_pretrained(peft_model_id)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='/home/ec2-user/SageMaker/models/kullm-polyglot-12-8b-v2', revision=None, task_type='CAUSAL_LM', inference_mode=True, r=8, target_modules={'xxx', 'query_key_value'}, lora_alpha=32, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={})

In [23]:
# 경고 메시지를 출력할 조건 확인
expected_path = '/home/ec2-user/SageMaker/models/kullm-polyglot-12-8b-v2'
if config.base_model_name_or_path != expected_path:
    # 조건이 충족될 때 출력할 경고 메시지
    print("경고: base_model_name_or_path가 예상된 경로와 다릅니다.")
    print(f"현재 경로: {config.base_model_name_or_path}")
    print(f"예상 경로: {expected_path}")
else :
    print(f"base_model_name_or_path 설정이 정상입니다. : {config.base_model_name_or_path}")

base_model_name_or_path 설정이 정상입니다. : /home/ec2-user/SageMaker/models/kullm-polyglot-12-8b-v2


In [24]:

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

### 로컬 모델 호출 테스트

In [25]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [26]:
gen('프로그래밍을 잘하기 위한 세 가지 방법은?')

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:430: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 프로그래밍을 잘하기 위한 세 가지 방법은?

### 답변:
세 가지 방법은 모두 프로그래밍 실력을 향상시키는 데 도움이 될 수 있습니다. 다음은 고려할 수 있는 몇 가지입니다:
1. 코드 리뷰: 코드 리뷰를 할 때는 다른 개발자, 팀 동료 또는 다른 소프트웨어 엔지니어에게 피드백을 제공해야 합니다. 피드백을 제공하면 다른 개발자가 나를 위해 코드를 개선하거나 코딩 방식을 바꾸도록 영감을 줄 수 있습니다. 또한 피드백은 잘못된 코드나 문제를 진단 및 수정하는 데 도움이 될 수 있습니다.
2. 좋은 코딩 표준을 준수하세요: 좋은 코딩 표준은 소프트웨어 코드를 잘 작성하는 데 있어 중요한 기초입니다. 다음은 좋은 코딩 표준에 대한 몇 가지 일반적인 가이드라인입니다:
- 간결성: 가능한 한 간단하게 작성하세요. 이렇게 하면 코드를 이해하고 유지 관리하기가 더 쉬워집니다.
변국: 변수는 적은 수의 정수나 문자열이어야 하고 다른 타입의 변수나 복합 배열은 허용되지 않습니다.	- 테스트: 프로그래머는 다양한 데이터 유형과 시나리오에서 작동하는지 확인하기 위해 테스트를 작성해야 합니다. 코딩 표준만 준수했다면 테스트는 필요하지 않습니다.
- 오류 처리: 오류가 발생하면 프로그래머는 명확하고 이해하기 쉬운 메시지를 통해 오류를 해결해야 합니다. 이렇게 하면 다른 개발자가 오류를 해결하고 코드를 명확하게 유지하는 데 도움이 됩니다.
3. 좋은 디버깅: 코드를 작성한 후에는 의도된 대로 작동하는지 확인하기 위해 디버깅해야 합니다. 이렇게 하면 소프트웨어에 문제가 발생하기 전에 문제를 식별하고 해결할 수 있습니다. 디버깅을 할 때는 코드에 대해 꼼꼼히 살펴보고 모든 버그 표시기에 도달해야 합니다. 디버그 관리자를 사용하거나 프로젝트 관리자를 사용함으로써 프로그래머는 코딩에 전념하면서 디버그 프로세스를 간소화할 수 있습니다. 디버깅은 코딩의 필수 단계이니 시간을 할애해야 한다는 점을 잊지 마세요.<|endoftext|>


## SageMaker Endpoint를 위한 함수 처리 테스트

In [28]:

import os.path as osp
import json
from typing import Union

def load_template(template_name: str):
    file_name = osp.join("../templates", f"{template_name}.json")
    if not osp.exists(file_name):
        raise ValueError(f"Can't read {file_name}")
    with open(file_name, encoding='utf-8') as fp:  # 'utf-8' 인코딩으로 파일을 엽니다.
        return json.load(fp)


def generate_prompt(template, instruction: str, input_text: Union[None, str] = None, label: Union[None, str] = None) -> str:
    if input_text:
        res = template["prompt_input"].format(instruction=instruction, input=input_text)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)
    if label:
        res = f"{res}{label}"
    return res

def get_payload(instruction, input_text, params, template_name="kullm"):
    
    template = load_template(template_name)
    prompt = generate_prompt(template, instruction, input_text)
    payload = {
        'inputs': prompt,
        'parameters': params
    }
    payload_str = json.dumps(payload)
    return payload_str.encode("utf-8")

def infer(self, payload, content_type="application/json", verbose=True):
    response = self.smr_client.invoke_endpoint(
        EndpointName=self.endpoint_name,
        ContentType=content_type,
        Body=payload
    )

    res = json.loads(response['Body'].read().decode("utf-8"))
    generated_text = res[0]["generated_text"]
    #generated_text = self.prompter.get_response(generated_text)

    generated_text = generated_text.split('###')[0]
    if verbose:
        pprint.pprint(f'Response: {generated_text}')
    return generated_text
def parse_response(query_response):
    
    def traverse(o, tree_types=(list, tuple)):
        if isinstance(o, tree_types):
            for value in o:
                for subvalue in traverse(value, tree_types):
                    yield subvalue
        else:
            yield o

    data = eval(query_response)
    
    listRes = []
    for value in traverse(data):
        listRes.append(value["generated_text"])
        
    if len(listRes) >= 2: return listRes
    else: return listRes[0].strip()

In [29]:
params = {
    "do_sample": True,
    "max_new_tokens": 512,
    "temperature": 0.7,
    "top_p": 0.9,
    "return_full_text": False,
    "repetition_penalty": 1.1,
    "presence_penalty": None,
    "eos_token_id": 2,
}
instruction = "다음 글을 알기 쉽게 요약해 주세요."
context = """
대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.
"""
payload = get_payload(instruction, context, params, "kullm")

In [30]:
data = json.loads(payload.decode('utf-8'))
input_prompt, params = data["inputs"], data["parameters"]

In [31]:
data

{'inputs': '아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\n',
 'parameters': {'do_sample': True,
  'max_new_tokens': 512,
  't

In [32]:
input_prompt

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\n'

In [33]:
result = model.generate(
        **tokenizer(
            input_prompt, 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to('cuda'), 
        max_new_tokens=512,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [34]:
#print(tokenizer.decode(result[0]))
#print(result)

decoded_output = tokenizer.decode(result[0])

print("decode_output : ", decoded_output)
# 이제 JSON 형식으로 변환
json_data = json.dumps({"generated_text": decoded_output})

decode_output :  아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.

### 명령어:
다음 글을 알기 쉽게 요약해 주세요.

### 입력:

대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. 
대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. 
이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 
특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. 
파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. 
파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.


### 응답:
이 글이 말하고자 하는 내용은 대규모 언어 모델에 대한 발전과 그들이 제공하는 인사이트입니다. 이러한 모델은 훈련되지 않은 작업에도 일반화되

In [35]:
json_data

'{"generated_text": "\\uc544\\ub798\\ub294 \\uc791\\uc5c5\\uc744 \\uc124\\uba85\\ud558\\ub294 \\uba85\\ub839\\uc5b4\\uc640 \\ucd94\\uac00 \\ucee8\\ud14d\\uc2a4\\ud2b8\\ub97c \\uc81c\\uacf5\\ud558\\ub294 \\uc785\\ub825\\uc774 \\uc9dd\\uc744 \\uc774\\ub8e8\\ub294 \\uc608\\uc81c\\uc785\\ub2c8\\ub2e4. \\uc694\\uccad\\uc744 \\uc801\\uc808\\ud788 \\uc644\\ub8cc\\ud558\\ub294 \\uc751\\ub2f5\\uc744 \\uc791\\uc131\\ud558\\uc138\\uc694.\\n\\n### \\uba85\\ub839\\uc5b4:\\n\\ub2e4\\uc74c \\uae00\\uc744 \\uc54c\\uae30 \\uc27d\\uac8c \\uc694\\uc57d\\ud574 \\uc8fc\\uc138\\uc694.\\n\\n### \\uc785\\ub825:\\n\\n\\ub300\\uaddc\\ubaa8 \\uc5b8\\uc5b4 \\ubaa8\\ub378(LLM; Large Language Models) \\ubd84\\uc57c\\uc758 \\ubc1c\\uc804\\uacfc LLM\\uc774 \\uac00\\uce58 \\uc788\\ub294 \\uc778\\uc0ac\\uc774\\ud2b8\\ub97c \\uc81c\\uacf5\\ud558\\ub294 \\ubb38\\uc81c \\uc138\\ud2b8 \\uc218\\uac00 \\uacc4\\uc18d \\uc99d\\uac00\\ud558\\uace0 \\uc788\\ub2e4\\ub294 \\uc18c\\uc2dd\\uc744 \\ub4e4\\uc5b4\\ubcf4\\uc168\\uc744 \

In [36]:
generated_text = parse_response(json_data)

In [37]:
generated_text

'아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요.\n\n### 명령어:\n다음 글을 알기 쉽게 요약해 주세요.\n\n### 입력:\n\n대규모 언어 모델(LLM; Large Language Models) 분야의 발전과 LLM이 가치 있는 인사이트를 제공하는 문제 세트 수가 계속 증가하고 있다는 소식을 들어보셨을 겁니다. \n대규모 데이터 세트와 여러 작업을 통해 훈련된 대규모 모델은 훈련되지 않은 특정 작업에도 잘 일반화됩니다. 이러한 모델을 파운데이션 모델(foundation model)이라고 하며, 스탠포드 HAI 연구소(Stanford Institute for Human-Centered Artificial Intelligence)에서 처음 대중화한 용어입니다. \n이러한 파운데이션 모델은 프롬프트 엔지니어링(prompt engineering) 기법의 도움으로 잘 활용할 수 있지만, 유스케이스가 도메인에 매우 특화되어 있거나 작업의 종류가 매우 다양하여 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. \n특정 도메인이나 작업에 대한 대규모 모델의 성능을 개선하기 위한 한 가지 접근 방식은 더 작은 작업별 데이터 세트로 모델을 추가로 훈련하는 것입니다. 파인 튜닝(fine-tuning)으로 알려진 이 접근 방식은 LLM의 정확도를 성공적으로 개선하지만, 모든 모델 가중치를 수정해야 합니다. \n파인 튜닝 데이터 세트 크기가 훨씬 작기 때문에 사전 훈련(pre-training)하는 것 보다 훨씬 빠르지만 여전히 상당한 컴퓨팅 성능과 메모리가 필요합니다. \n파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 비용이 많이 들고 원본 모델과 동일한 크기의 모델을 생성하므로 스토리지 용량에도 부담이 됩니다.\n\n\n### 응답:\n이 글이 말하고자 하는 내용은 대규모 언어 모델에 대한 발전과 그들이 제공하는 인사이트입니다. 이러한 모델은 훈련되지 않은 작업에도 일반화되

In [38]:
parts = generated_text.split('### 응답:')

# parts의 길이가 2 이상이면, '### 응답:' 이후의 부분을 추출합니다.
if len(parts) >= 2:
    response_text = parts[1]
else:
    response_text = ""  # '### 응답:'이 없는 경우 빈 문자열을 반환합니다.

print(response_text.strip())  # 앞뒤 공백을 제거하고 출력합니다.

이 글이 말하고자 하는 내용은 대규모 언어 모델에 대한 발전과 그들이 제공하는 인사이트입니다. 이러한 모델은 훈련되지 않은 작업에도 일반화되고, 매우 다양한 문제에 적용될 수 있습니다. 하지만 특정 도메인이나 작업에 대해 프롬프트 엔지니어링을 하려면 많은 노력이 필요하고 모델을 추가로 커스터마이징해야 하는 경우가 많습니다. 더 작은 작업에 대해 LLM 모델을 추가로 훈련하면 정확도를 높일 수 있지만, 많은 컴퓨팅 및 메모리 리소스가 필요합니다. 파인 튜닝은 원본 모델의 모든 파라미터 가중치를 수정하므로 효율적이지만 스토리지, 컴퓨팅 및 메모리 용량을 많이 사용합니다.<|endoftext|>
